In [5]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fe

In [6]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-02-23 02:55:01--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.25MB/s    in 0.7s    

2023-02-23 02:55:03 (1.25 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [7]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [8]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
health_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
health_df.show()



+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     650634| R3EQSTM9PWRAL|B0091LBZSU|     578484426|Demograss Capsule...|Health & Personal...|          3|            0|          0|   N|                Y|         Three Stars|Only came with 30...| 2015-08-31|
|         US|   19827510| RBWPRK17XKIXD|B00PWW3LQ6|     456433146|Viva Labs #1 Prem...|Health & Personal

In [9]:
# Get the number of rows in the DataFrame.
health_df.count()

5331449

# Transform the Data

## Create the "review_id_table".

In [10]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df= health_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R3EQSTM9PWRAL|     650634|B0091LBZSU|     578484426| 2015-08-31|
| RBWPRK17XKIXD|   19827510|B00PWW3LQ6|     456433146| 2015-08-31|
| RRSLOAF273XFC|    1520474|B00DKEWA92|     460764511| 2015-08-31|
|R3S8W9Q6SWIT8O|   23905905|B0015R3A7M|     135102038| 2015-08-31|
|R3QQ6NSLRVBFJC|   28215779|B006B8U8BG|     200666829| 2015-08-31|
|R2XYDBMHUVJCSX|   23260912|B00PFZFD8Y|     344168617| 2015-08-31|
|R1L94ESVC657A9|    7965616|B0034792JG|     291831570| 2015-08-31|
|R3DI4B8LDWFQ3K|    3443178|B000052XIA|     493678550| 2015-08-31|
|R1G7VV4HCXUQQL|   20838781|B004GCUK8A|       3902598| 2015-08-31|
|R3TTNYN1TAMUSU|   15267517|B00K7W1QAO|     633971374| 2015-08-31|
|R3AMX2D9HMWV6Z|   30105089|B0034KYA36|     854858899| 2015-08-31|
|R3PO3K321GY8HI|    1622207|B00C7UZTT8|     264608272| 2015-08

## Create the "products" Table

In [11]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df=health_df.select(["product_id", "product_title"]).drop_duplicates(["product_id"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0000045284|Loreal Paris Kaja...|
|0000800309|Himalaya Clarina ...|
|0004124677|  The Workbench Book|
|0004720962|    SAS Self-Defense|
|0006386830|Leslie Nielsen's ...|
|0006726909|Spanish Large Pri...|
|0007223099|Rani and the Thre...|
|0007241887|The Return to Nar...|
|0007622880|                Mine|
|0021500339|                Ohio|
|0064401251|It Can't Hurt For...|
|0072884975|THERMODYNAMICS-PR...|
|0077107950|Services Marketin...|
|0077663004|Contemporary Nutr...|
|0077676920|Contemporary Nutr...|
|0077862333|Auditing and Assu...|
|0091920655|     The Ghost Agent|
|0099252716|The Voyage of the...|
|0099274175|   Shadows of Empire|
|0140018654|Exile and the Kin...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [12]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df=health_df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    4919528|             5|
|   51451778|             3|
|   12713799|             1|
|    8673341|             1|
|   42146698|             2|
|    1117644|             2|
|   28058398|             1|
|   14375645|             1|
|   24540309|             1|
|   39715602|             1|
|     654272|             1|
|    9015608|             5|
|   43920023|            42|
|   38209321|             1|
|   12328685|             1|
|    2866605|             2|
|   38273165|             5|
|   41066514|             1|
|   28377689|             2|
|   17018784|             2|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [13]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df= health_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| R3EQSTM9PWRAL|          3|            0|          0|   N|
| RBWPRK17XKIXD|          5|            0|          0|   N|
| RRSLOAF273XFC|          5|            1|          2|   N|
|R3S8W9Q6SWIT8O|          4|            0|          0|   N|
|R3QQ6NSLRVBFJC|          4|            0|          0|   N|
|R2XYDBMHUVJCSX|          2|            0|          1|   N|
|R1L94ESVC657A9|          5|            0|          0|   N|
|R3DI4B8LDWFQ3K|          1|            3|          4|   N|
|R1G7VV4HCXUQQL|          5|            0|          0|   N|
|R3TTNYN1TAMUSU|          5|            0|          0|   N|
|R3AMX2D9HMWV6Z|          5|            0|          0|   N|
|R3PO3K321GY8HI|          1|            0|          0|   N|
| RVQ0UNYX6IEVK|          1|            0|          1|   N|
|R25E9NIGO2XQ3D|          5|            

# Load

In [14]:
import getpass
password = getpass.getpass()



mode = "append"
jdbc_url="jdbc:postgresql://database-1.c6it5lbg2swj.us-east-2.rds.amazonaws.com:5432/postgres"
config = {"user":"root", "password":password, "driver":"org.postgresql.Driver"}

··········


In [16]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table_health', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products_health', mode=mode, properties=config)

In [15]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers_health', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vines_table_health', mode=mode, properties=config)